In [1]:
import joblib
import pandas as pd

# Load saved model and scaler
rf_model = joblib.load('random_forest_air_quality.pkl')
scaler = joblib.load('scaler_air_quality.pkl')


In [2]:
# Example new sensor reading
new_data = pd.DataFrame({
    'mq3_rs_r0': [0.45],  # Replace with new Rs/R0 value from MQ-3
    'mq9_rs_r0': [0.52],  # Replace with new Rs/R0 value from MQ-9
    'hour': [18]          # Hour of the day (0–23)
})


In [3]:
# Scale the input data
new_data_scaled = scaler.transform(new_data)


In [4]:
#make predictions
predicted_label = rf_model.predict(new_data_scaled)[0]
print("Predicted Air Quality Condition:", predicted_label)


Predicted Air Quality Condition: Foul


In [5]:
# getting probablity of each classes
probabilities = rf_model.predict_proba(new_data_scaled)
print("Class Probabilities:", probabilities)


Class Probabilities: [[0.         0.56333333 0.         0.43666667]]


In [8]:
import pandas as pd
from datetime import datetime
import joblib

# Load model and scaler
rf_model = joblib.load('random_forest_air_quality.pkl')
scaler = joblib.load('scaler_air_quality.pkl')

# Sensor R0 values (from calibration)
mq3_r0 = 17200
mq9_r0 = 21940

# Rs/R0 function
def rs_r0(adc, r0, vcc=5.0, rl=5000, adc_vmax=3.3):
    vout = (adc / 4095) * adc_vmax
    if vout == 0:
        return float('inf')
    rs = rl * ((vcc / vout) - 1)
    return rs / r0

# 🔥 Firebase Data
firebase_data = {
    'mq3_value': 1235,
    'mq9_value': 971,
    'timestamp': "2025-06-10 10:40:41"
}

# Extract hour
timestamp_obj = datetime.strptime(firebase_data['timestamp'], "%Y-%m-%d %H:%M:%S")
hour = timestamp_obj.hour

# Convert ADC to Rs/R0
mq3_rs = rs_r0(firebase_data['mq3_value'], mq3_r0)
mq9_rs = rs_r0(firebase_data['mq9_value'], mq9_r0)

# Build DataFrame for model
new_data = pd.DataFrame([{
    'mq3_rs_r0': mq3_rs,
    'mq9_rs_r0': mq9_rs,
    'hour': hour
}])

# Scale and predict
new_data_scaled = scaler.transform(new_data)
prediction = rf_model.predict(new_data_scaled)[0]

print("Prediction:", prediction)


Prediction: Clean
